# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 36 new papers today
          19 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/18 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2302.08523


extracting tarball to tmp_2302.08523...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.08532


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2302.08523/mnras_template.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

extracting tarball to tmp_2302.08962...

 done.


Retrieving document from  https://arxiv.org/e-print/2302.09032


extracting tarball to tmp_2302.09032...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.08628-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.08628) | **New measurement of the diffusion of carbon dioxide on non-porous  amorphous solid water**  |
|| <mark>Jiao He</mark>, et al. -- incl., <mark>Tushar Suhasaria</mark>, <mark>Thomas K. Henning</mark> |
|*Appeared on*| *2023-02-20*|
|*Comments*| *13 pages, 6 figures*|
|**Abstract**| The diffusion of molecules on interstellar grain surfaces is one of the most important driving forces for the molecular complexity in the interstellar medium. Due to the lack of laboratory measurements, astrochemical modeling of grain surface processes usually assumes a constant ratio between the diffusion energy barrier and the desorption energy. This over-simplification inevitably causes large uncertainty in model predictions. We present a new measurement of the diffusion of CO$_2$ molecules on the surface of non-porous amorphous solid water (np-ASW), an analog of the ice mantle that covers cosmic dust grains. A small coverage of CO$_2$ was deposited onto an np-ASW surface at 40~K, the subsequent warming of the ice activated the diffusion of CO$_2$ molecules, and a transition from isolated CO$_2$ to CO$_2$ clusters was seen in the infrared spectra. To obtain the diffusion energy barrier and pre-exponential factor simultaneously, a set of isothermal experiments were carried out. The values for the diffusion energy barrier and pre-exponential factor were found to be $1300\pm110$~K and $10^{7.6\pm0.8}$~s$^{-1}$. A comparison with prior laboratory measurements on diffusion is discussed. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.08736-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.08736) | **CRIRES+ detection of CO emissions lines and temperature inversions on  the dayside of WASP-18b and WASP-76b**  |
|| F. Yan, et al. -- incl., <mark>U. Heiter</mark> |
|*Appeared on*| *2023-02-20*|
|*Comments*| *11 pages, 12 figures; accepted for publication in A&A*|
|**Abstract**| The dayside atmospheres of ultra-hot Jupiters (UHJs) are predicted to possess temperature inversion layers with extremely high temperatures at high altitudes. We observed the dayside thermal emission spectra of WASP-18b and WASP-76b with the new CRIRES+ high-resolution spectrograph at near-infrared wavelengths. Using the cross-correlation technique, we detected strong CO emission lines in both planets, which confirms the existence of temperature inversions on their dayside hemispheres. The two planets are the first UHJs orbiting F-type stars with CO emission lines detected; previous detections were mostly for UHJs orbiting A-type stars. Evidence of weak H2O emission signals is also found for both planets. We further applied forward-model retrievals on the detected CO lines and retrieved the temperature-pressure profiles along with the CO volume mixing ratios. The retrieved logarithmic CO mixing ratio of WASP-18b (-2.2) is slightly higher than the value predicted by the self-consistent model assuming solar abundance. For WASP-76b, the retrieved CO mixing ratio (-3.6) is broadly consistent with the value of solar abundance. In addition, we included the equatorial rotation velocity (Veq ) in the retrieval when analyzing the line profile broadening. The obtained Veq is 7.0 km/s for WASP-18b and 5.2 km/s for WASP-76b, which are consistent with the tidally locked rotational velocities. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.08962-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.08962) | **The Hubble PanCET program: The near-ultraviolet transmission spectrum of  WASP-79b**  |
|| A. Gressier, et al. -- incl., <mark>G.W. Henry</mark>, <mark>T. Mikal-Evans</mark> |
|*Appeared on*| *2023-02-20*|
|*Comments*| *Accepted for publication January 31, 2023 in the Journal Astronomy & Astrophysics*|
|**Abstract**| We present Hubble Space Telescope (HST) transit observations of the Hot-Jupiter WASP-79b acquired with the Space Telescope Imaging Spectrograph (STIS) in the near ultraviolet (NUV). Two transit observations, part of the PanCET program, are used to obtain the transmission spectra of the planet between 2280 and 3070{\AA}. We correct for systematic effects in the raw data using the jitter engineering parameters and polynomial modelling to fit the white light curves of the two transits. We observe an increase in the planet-to-star radius ratio at short wavelengths, but no spectrally resolved absorption lines. The difference between the radius ratios at 2400 and 3000{\AA} reaches $0.0191\pm0.0042$ ($\sim$4.5$-\sigma$). Although the NUV transmission spectrum does not show evidence of hydrodynamical escape, the strong atmospheric features are likely due to species at very high altitudes. We performed a 1D simulation of the temperature and composition of WASP-79b using Exo-REM. The temperature pressure profile crosses condensation curves of radiatively active clouds, particularly MnS, Mg$_2$SiO$_4$, Fe, and Al$_2$O$_3$. Still, none of these species produces the level of observed absorption at short wavelengths and can explain the observed increase in the planet's radius. WASP-79b's transit depth reaches 23 scale height, making it one of the largest spectral features observed in an exoplanet at this temperature ($\sim$1700 K). The comparison of WASP-79b's transmission spectrum with three warmer hot Jupiters shows a similar level of absorption to WASP-178b and WASP-121b between 0.2 and 0.3$\mu$m, while HAT-P-41b's spectrum is flat. The features could be explained by SiO absorption. |

## Failed papers

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.08889-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.08889) | **Model-independent measurement of cosmic curvature with the latest $H(z)$  and SNe Ia data: A comprehensive investigation**  |
|| <mark>Jing-Zhao Qi</mark>, Ping Meng, <mark>Jing-Fei Zhang</mark>, <mark>Xin Zhang</mark> |
|*Appeared on*| *2023-02-20*|
|*Comments*| *9 pages, 3 figures*|
|**Abstract**| In the context of the discrepancies between the early and late universe, we emphasize the importance of independent measurements of the cosmic curvature in the late universe. We present an investigation of the model-independent measurement of the cosmic curvature parameter $\Omega_k$ in the late universe with the latest Hubble parameter $H(z)$ measurements and type Ia supernovae (SNe Ia) data. For that, we use two reconstruction methods, the Gaussian process (GP) and artificial neural network (ANN) methods, to achieve the distance construction from $H(z)$ data. In the results obtained by different combinations of observations and reconstruction methods, the tightest constraint on the cosmic curvature is $\Omega_k=-0.03\pm0.11$, in good agreement with zero curvature. This result is the most precise constraint on the cosmic curvature obtained among the recent related estimations. Our findings suggest that the observational data of the late universe support a flat universe. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.08926-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.08926) | **LISC Catalog of Open Clusters.III. 83 Newly found Galactic disk open  clusters using Gaia EDR3**  |
|| Huanbin Chi, Feng Wang, <mark>Zhongmu Li</mark> |
|*Appeared on*| *2023-02-20*|
|*Comments*| *15 pages, 6 figures, Accepted by RAA*|
|**Abstract**| As groups of coeval stars born from the same molecular cloud, an Open cluster (OC) is an ideal laboratory for studying the structure and dynamical evolution of the Milky Way. The release of High-Precision Gaia Early Data Release 3 (Gaia EDR3) and modern machine-learning methods offer unprecedented opportunities to identify OCs. In this study, we extended conventional HDBSCAN (e-HDBSCAN) for searching for new OCs in Gaia EDR3. A pipeline was developed based on the parallel computing technique to blindly search for open clusters from Gaia EDR3 within Galactic latitudes $\left| b \right|$ $<$25 $^\circ$. As a result, we obtained 3787 star clusters, of which 83 new OCs were reported after cross-match and visual inspection. At the same time, the main star cluster parameters are estimated by colour-magnitude diagram fitting. The study significantly increases the sample size and physical parameters of open clusters in the catalogue of OCs. It shows the incompleteness of the census of OCs across our Galaxy. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.08960-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.08960) | **A z=1.85 galaxy group in CEERS: evolved, dustless, massive Intra-Halo  Light and a Brightest Group Galaxy in the making**  |
|| Rosemary T. Coogan, et al. -- incl., <mark>Steven L. Finkelstein</mark> |
|*Appeared on*| *2023-02-20*|
|*Comments*| *14 pages + appendix, 11 figures, 4 tables, submitted to A&A*|
|**Abstract**| We present CEERS JWST/NIRCam imaging of a massive galaxy group at z=1.85, to explore the early JWST view on massive group formation in the distant Universe. The group contains >16 members (including 6 spectros. confirmations) down to log10(Mstar/Msun)=8.5, including the Brightest Group Galaxy (BGG) in the process of actively assembling at this redshift. The BGG is comprised of multiple merging components extending ~3.6" (30kpc) across the sky. The BGG contributes 69% of the group's total galactic stellar mass, with one of the merging components containing 76% of the total mass of the BGG and a SFR>1810Msun/yr. Most importantly, we detect intra-halo light (IHL) in several HST and JWST/NIRCam bands, allowing us to construct a state-of-the-art rest-frame UV-NIR Spectral Energy Distribution of the IHL for the first time at this high redshift. This allows stellar population characterisation of both the IHL and member galaxies, as well as the morphology distribution of group galaxies vs. their star-formation activity when coupled with Herschel data. We create a stacked image of the IHL, giving us a sensitivity to extended emission of 28.5 mag/arcsec2 at rest-frame 1um. We find that the IHL is extremely dust poor (Av~0), containing an evolved stellar population of log10(t50/yr)=8.8, corresponding to a formation epoch for 50% of the stellar material 0.63Gyr before z=1.85. There is no evidence of ongoing star-formation in the IHL. The IHL in this group at z=1.85 contributes ~10% of the total stellar mass, comparable with what is observed in local clusters. This suggests that the evolution of the IHL fraction is more self-similar with redshift than predicted by some models, challenging our understanding of IHL formation during the assembly of high-redshift clusters. JWST is unveiling a new side of group formation at this redshift, which will evolve into Virgo-like structures in the local Universe. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.09032-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.09032) | **The Ups and Downs of Early Dark Energy solutions to the Hubble tension:  a review of models, hints and constraints circa 2023**  |
|| Vivian Poulin, <mark>Tristan L. Smith</mark>, Tanvi Karwal |
|*Appeared on*| *2023-02-20*|
|*Comments*| *46 + 3 pages, 28 + 3 figures. Submitted as an invited review to Physics of the Dark Universe. Comments welcome!*|
|**Abstract**| We review the current status of Early Dark Energy (EDE) models proposed to resolve the ``Hubble tension'', the discrepancy between ``direct'' measurements of the current expansion rate of the Universe and ``indirect measurements'' for which the values inferred rely on the $\Lambda$CDM cosmological model calibrated on early-universe data. EDE refers to a new form of dark energy active at early times (typically a scalar-field), that quickly dilutes away at a redshift close to matter-radiation equality. The role of EDE is to decrease the sound horizon by briefly contributing to the Hubble rate in the pre-recombination era. We summarize the results of several analyses of EDE models suggested thus far in light of recent cosmological data, including constraints from the canonical {\it Planck} data, baryonic acoustic oscillations and Type Ia supernovae, and the more recent hints driven by cosmic microwave background observations using the Atacama Cosmology Telescope. We also discuss potential challenges to EDE models, from theoretical ones (a second ``cosmic coincidence'' problem in particular) to observational ones, related to the amplitude of clustering on scales of $8h$/Mpc as measured by weak-lensing observables (the so-called $S_8$ tension) and the galaxy power spectrum from BOSS analyzed through the effective field theory of large-scale structure. We end by reviewing recent attempts at addressing these shortcomings of the EDE proposal. While current data remain inconclusive on the existence of an EDE phase, we stress that given the signatures of EDE models imprinted in the CMB and matter power spectra, next-generation experiments can firmly establish whether EDE is the mechanism responsible for the Hubble tension and distinguish between the various models suggested in the literature. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.08657-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.08657) | **Late Accretion of Ceres-like Asteroids and Their Implantation into the  Outer Main Belt**  |
|| Driss Takir, et al. -- incl., <mark>Wladimir Neumann</mark> |
|*Appeared on*| *2023-02-20*|
|*Comments*| **|
|**Abstract**| Low-albedo asteroids preserve a record of the primordial solar system planetesimals and the conditions in which the solar nebula was active. However, the origin and evolution of these asteroids are not well-constrained. Here we measured visible and near-infrared (0.5 - 4.0 microns) spectra of low-albedo asteroids in the mid-outer main belt. We show that numerous large (d > 100 km) and dark (geometric albedo < 0.09) asteroids exterior to the dwarf planet Ceres' orbit share the same spectral features, and presumably compositions, as Ceres. We also developed a thermal evolution model that demonstrates that these Ceres-like asteroids have highly-porous interiors, accreted relatively late at 1.5 - 3.5 Myr after the formation of calcium-aluminum-rich inclusions, and experienced maximum interior temperatures of < 900 K. Ceres-like asteroids are localized in a confined heliocentric region between 3.0 - 3.4 au but were likely implanted from more distant regions of the solar system during the giant planet's dynamical instability. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2302.08628.md
    + _build/html/tmp_2302.08628/./fitting.png
    + _build/html/tmp_2302.08628/./bandArea.png
    + _build/html/tmp_2302.08628/./arrh.png
exported in  _build/html/2302.08736.md
    + _build/html/tmp_2302.08736/./SYSREM.png
    + _build/html/tmp_2302.08736/./SYSREM-H2O.png
    + _build/html/tmp_2302.08736/./spectrum-demo.png
exported in  _build/html/2302.08962.md
    + _build/html/tmp_2302.08962/./WASP79_wlc.png
    + _build/html/tmp_2302.08962/./Fig9a_revised.png
    + _build/html/tmp_2302.08962/./Fig9b_revised2.png
    + _build/html/tmp_2302.08962/./Fig6_revised.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand$</div>



<div id="title">

# New measurement of the diffusion of carbon dioxide on non-porous amorphous solid water

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.08628-b31b1b.svg)](https://arxiv.org/abs/2302.08628)<mark>Appeared on: 2023-02-20</mark> - _13 pages, 6 figures_

</div>
<div id="authors">

<mark><mark>Jiao He</mark></mark>, et al. -- incl., <mark><mark>Tushar Suhasaria</mark></mark>, <mark><mark>Thomas K. Henning</mark></mark>

</div>
<div id="abstract">

**Abstract:** The diffusion of molecules on interstellar grain surfaces is one of the most important driving forces for the molecular complexity in the interstellar medium. Due to the lack of laboratory measurements, astrochemical modeling of grain surface processes usually assumes a constant ratio between the diffusion energy barrier and the desorption energy. This over-simplification inevitably causes large uncertainty in model predictions. We present a new measurement of the diffusion of CO$_2$molecules on the surface of non-porous amorphous solid water (np-ASW), an analog of the ice mantle that covers cosmic dust grains. A small coverage of CO$_2$was deposited onto an np-ASW surface at 40 K, the subsequent warming of the ice activated the diffusion of CO$_2$molecules, and a transition from isolated CO$_2$to CO$_2$clusters was seen in the infrared spectra. To obtain the diffusion energy barrier and pre-exponential factor simultaneously, a set of isothermal experiments were carried out. The values for the diffusion energy barrier and pre-exponential factor were found to be$1300\pm110$K and$10^{7.6\pm0.8}$s$^{-1}$. A comparison with prior laboratory measurements on diffusion is discussed.

</div>

<div id="div_fig1">

<img src="tmp_2302.08628/./fitting.png" alt="Fig2" width="100%"/>

**Figure 2. -** Selected RAIRS spectra in the range of the CO$_2$$\nu_3$ mode during the isothermal experiment at different temperatures. Each panel depicts the CO$_2$ absorption peak at various times since the start of an isothermal experiment. The time and temperature of the isotherm are indicated in the figure. The black curves represent the experimental data, while the orange curves represent the fittings made with a sum of two Gaussian functions. (*fig:fitting*)

</div>
<div id="div_fig2">

<img src="tmp_2302.08628/./bandArea.png" alt="Fig3" width="100%"/>

**Figure 3. -** The band areas of the 2330 and 2355 cm$^{-1}$ components of the CO$_2$ absorption peak during the isothermal evolution at different temperatures. (*fig:bandArea*)

</div>
<div id="div_fig3">

<img src="tmp_2302.08628/./arrh.png" alt="Fig5" width="100%"/>

**Figure 5. -** Arrhenius plot $log(k_D)$ over $1/T$, where $k_D$ is the diffusion rate, and $T$ is the isotherm temperature. The linear fitting is shown in orange.  (*fig:arrh*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$</div>



<div id="title">

#  detection of CO emissions lines and temperature inversions on the dayside of WASP-18b and WASP-76b

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.08736-b31b1b.svg)](https://arxiv.org/abs/2302.08736)<mark>Appeared on: 2023-02-20</mark> - _11 pages, 12 figures; accepted for publication in A&A_

</div>
<div id="authors">

F. Yan, et al. -- incl., <mark>U. Heiter</mark>

</div>
<div id="abstract">

**Abstract:** The dayside atmospheres of ultra-hot Jupiters (UHJs) are predicted to possess temperature inversion layers with extremely high temperatures at high altitudes. We observed the dayside thermal emission spectra of WASP-18b and WASP-76b with the new CRIRES$^+$high-resolution spectrograph at near-infrared wavelengths. Using the cross-correlation technique, we detected strong CO emission lines in both planets, which confirms the existence of temperature inversions on their dayside hemispheres.The two planets are the first UHJs orbiting F-type stars with CO emission lines detected; previous detections were mostly for UHJs orbiting A-type stars.Evidence of weak$\mathrm{H_2O}$emission signals is also found for both planets. We further applied forward-model retrievals on the detected CO lines and retrieved the temperature-pressure profiles along with the CO volume mixing ratios. The retrieved logarithmic CO mixing ratio of WASP-18b ($-2.2_{-1.5}^{+1.4}$) is slightly higher than the value predicted by the self-consistent model assuming solar abundance. For WASP-76b, the retrieved CO mixing ratio ($-3.6_{-1.6}^{+1.8}$) is broadly consistent with the value of solar abundance. In addition, we included the equatorial rotation velocity ($\varv_\mathrm{eq}$) in the retrieval when analyzing the line profile broadening. The obtained$\varv_\mathrm{eq}$is$7.0\pm{2.9}$km s$^{-1}$for WASP-18b and$5.2_{-3.0}^{+2.5}$km s$^{-1}$for WASP-76b, which are consistent with the tidally locked rotational velocities.

</div>

<div id="div_fig1">

<img src="tmp_2302.08736/./SYSREM.png" alt="Fig1" width="100%"/>

**Figure 1. -** Detection significance of the CO signal with different \texttt{SYSREM} iteration numbers. The maximum S/N values are marked with the red triangles. The value here is measured at fixed $K_\mathrm{p}$ and at the $\mathrm{\Delta} \varv$ location where the detection signal is the strongest. (*App-sysrem*)

</div>
<div id="div_fig2">

<img src="tmp_2302.08736/./SYSREM-H2O.png" alt="Fig2" width="100%"/>

**Figure 2. -** Same as Fig. \ref{App-sysrem}, but for the $\mathrm{H_2O}$ signal. (*App-sysrem-H2O*)

</div>
<div id="div_fig3">

<img src="tmp_2302.08736/./spectrum-demo.png" alt="Fig3" width="100%"/>

**Figure 3. -** Example of the reduced CRIRES$^+$ spectrum of WASP-76 using the K2148 wavelength setting. The colors represent the three signal detectors. (*spec-demo*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$</div>



<div id="title">

# 

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.08962-b31b1b.svg)](https://arxiv.org/abs/2302.08962)<mark>Appeared on: 2023-02-20</mark> - _Accepted for publication January 31, 2023 in the Journal Astronomy & Astrophysics_

</div>
<div id="authors">

A. Gressier, et al. -- incl., <mark><mark>G.W. Henry</mark></mark>, <mark><mark>T. Mikal-Evans</mark></mark>

</div>
<div id="abstract">

**Abstract:** We present Hubble Space Telescope (HST) transit observations of the Hot-Jupiter WASP-79 b acquired with the Space Telescope Imaging Spectrograph (STIS) in the near ultraviolet (NUV). Two transit observations, part of the PanCET program, are used to obtain the transmission spectra of the planet between 2280 and 3070\text{\AA}.        We correct for systematic effects in the raw data using the jitter engineering parameters and polynomial modelling to fit the white light curves of the two transits.        We observe an increase in the planet-to-star radius ratio at short wavelengths, but no spectrally resolved absorption lines. The difference between the radius ratios at 2400\text{\AA}and 3000\text{\AA}reaches$0.0191\pm0.0042$($\sim$4.5$-\sigma$).        Although the NUV transmission spectrum does not show evidence of hydrodynamical escape,        the strong atmospheric features are likely due to species at very high altitudes. We performed a 1D simulation of the temperature and composition of WASP-79 b using Exo-REM.        The temperature pressure profile crosses condensation curves of radiatively active clouds, particularly MnS, Mg$_2$SiO$_4$, Fe, and Al$_2$O$_3$. Still, none of these species produces the level of observed absorption at short wavelengths and can explain the observed increase in the planet's radius.        WASP-79 b's transit depth reaches 23 scale height, making it one of the largest spectral features observed in an exoplanet at this temperature ($\sim$1700 K).        The comparison of WASP-79 b's transmission spectrum with three warmer hot Jupiters shows a similar level of absorption to WASP-178 b and WASP-121 b between 0.2 and 0.3$\mu$m, while HAT-P-41 b's spectrum is flat. The features could be explained by SiO absorption.

</div>

<div id="div_fig1">

<img src="tmp_2302.08962/./WASP79_wlc.png" alt="Fig9" width="100%"/>

**Figure 9. -** WASP-79b white light curves for visit \#65 (left) and \#66 (right). Top: Normalised raw light curves. Middle: Flux corrected from systematic errors and fitted with a transit model.
Bottom: Residuals between the flux corrected spectra, the best-fit models, and 2-$\rm \sigma$ error bars (dotted line). Visit \#65's raw data display a large ramp effect compared to visit \#66. This ramp is mainly correlated to the HST orbital phase. (*fig:1wlc*)

</div>
<div id="div_fig2">

<img src="tmp_2302.08962/./Fig9a_revised.png" alt="Fig6.1" width="50%"/><img src="tmp_2302.08962/./Fig9b_revised2.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** WASP-79b HST transmission spectrum observations (black) and simulated spectra using Exo-REM (colours). WASP-79 b atmosphere is simulated using a 10 x solar metallicity and an interior temperature of 600 K, and we include different clouds (top). We also tested a clear atmosphere while changing the metallicity of the atmosphere (bottom). (*fig6:exorem_simulations*)

</div>
<div id="div_fig3">

<img src="tmp_2302.08962/./Fig6_revised.png" alt="Fig3" width="100%"/>

**Figure 3. -** WASP-79b NUV transmission spectra in 4\text{\AA} bins for visit \#66. We indicate the overall NUV transmission spectrum (white light curve) R$_{\rm P}$/R$_{\rm\star}$(NUV) = 0.1059 in red, along with the planet-to-star radius ratio after 2500(\AA) in green R$_{\rm P}$/R$_{\rm\star}$(>2500\AA) = 0.1031.  (*fig:3spectralbins*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

211  publications in the last 7 days.
	 _build/html/2302.08962.md
	 _build/html/2302.08736.md
	 _build/html/2302.08628.md
	 _build/html/2302.07916.md
	 _build/html/2302.07880.md
	 _build/html/2302.07497.md
	 _build/html/2302.07277.md
	 _build/html/2302.07256.md
	 _build/html/2302.07234.md
	 _build/html/2302.07057.md
	 _build/html/2302.05694.md
	 _build/html/2302.05465.md
	 _build/html/2302.04507.md
	 _build/html/2302.04239.md
	 _build/html/2302.04138.md
	 _build/html/2302.03699.md
	 _build/html/2302.03576.md
	 _build/html/2302.03042.md
	 _build/html/2302.03039.md
	 _build/html/2302.02890.md
	 _build/html/2302.02611.md
	 _build/html/2302.02429.md
	 _build/html/2302.01678.md
	 _build/html/2302.00450.md
	 _build/html/2301.13766.md
	 _build/html/2301.13526.md
	 _build/html/2301.12890.md
	 _build/html/2301.11942.md
	 _build/html/2301.10717.md
	 _build/html/2301.10666.md
	 _build/html/2301.10496.md
	 _build/html/2301.09621.md
	 _build/html/2301.09140.md
	 _build/html/2301.08770.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

12  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> daily </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


# Debugging papers